In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from tqdm.auto import tqdm
from parameters import parse_args
import sys; sys.argv=['']; del sys

In [4]:
import os
from constants import (
    LLMsMap,
    EmbeddingModelsMap,
)

from api_models import set_llm_and_embed
args = parse_args()


In [5]:
args.embed_model = 'bge_m3'

llm_name = LLMsMap[args.llm]
embed_model_name = EmbeddingModelsMap[args.embed_model]
print(f"Using LLM: {llm_name}")
print(f"Using Embedding Model: {embed_model_name}")

set_llm_and_embed(
    llm_type=args.llm_type,
    llm_name=llm_name,
    embed_model_name=embed_model_name,
)

Using LLM: meta-llama/Meta-Llama-3-70B-Instruct
Using Embedding Model: BAAI/bge-m3


/Users/junaid/Anaconda/anaconda3/envs/ML/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/junaid/Anaconda/anaconda3/envs/ML/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
import pickle

def get_dataset_nodes(
    dataset_name,
    path: str = 'indexed_nodes',
    use_mc: bool = False,
    use_summary: bool = False
):
    indexed_nodes = pickle.load(open(f'{path}/{dataset_name}{"_mc" if use_mc else ""}{"_sm" if use_summary else ""}.pkl', 'rb'))
    print(f"Number of indexed nodes: {len(indexed_nodes)}")
    req_nodes = pickle.load(open(f'similar_requirements/{dataset_name}.pkl', 'rb'))
    print(f"Number of requirements nodes: {len(req_nodes)}")
    return indexed_nodes, req_nodes

# indexed_nodes, req_nodes = get_dataset_nodes(dataset_name, 'indexed_nodes')

In [7]:
import asyncio
import json
import os
import pickle
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.evaluation import SemanticSimilarityEvaluator
from llama_index.core import Settings
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.indices import VectorStoreIndex
from collections import defaultdict
from code2graph import CLASS_NAME_LABEL, get_docs_nxg
from typing import List
from typing import Union
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, Document
from constants import BM25_INDEX_RETREIVER, VECTOR_INDEX_RETREIVER
from evaluation import evaluate_response, get_solutions
from indexing.constants import (
    CLASS_NAME_LABEL,
    DOCSTRING_LABEL,
    ATTRIBUTES_LABEL,
    ATTRIBUTE_NAME_LABEL,
    ATTRIBUTES_TYPE_LABEL,
    METHOD_NAME_LABEL,
    METHODS_LABEL,
)
from llama_index.core import QueryBundle
from retrievers import get_reachable_nodes


def get_dataset_nodes(
    dataset_name,
    path: str = 'indexed_nodes',
    use_mc: bool = False,
    use_summary: bool = False
):
    indexed_nodes = pickle.load(open(f'{path}/{dataset_name}{"_mc" if use_mc else ""}{"_sm" if use_summary else ""}.pkl', 'rb'))
    print(f"Number of indexed nodes: {len(indexed_nodes)}")
    req_nodes = pickle.load(open(f'similar_requirements/{dataset_name}.pkl', 'rb'))
    print(f"Number of requirements nodes: {len(req_nodes)}")
    return indexed_nodes, req_nodes

def get_graph_node_str(graph_node):
    content = f"Class {graph_node[CLASS_NAME_LABEL]}\n"
    if graph_node[DOCSTRING_LABEL]:
        content += f"Docstring: {graph_node[DOCSTRING_LABEL]}\n"
        
    # if ATTRIBUTES_LABEL in graph_node and len(graph_node[ATTRIBUTES_LABEL]):
    #     content += f"Attributes: \n"
    #     for attr in graph_node[ATTRIBUTES_LABEL]:
    #         content += f"{attr[ATTRIBUTE_NAME_LABEL]}: {attr[ATTRIBUTES_TYPE_LABEL]}\n"
    
    # method_names = [i[METHOD_NAME_LABEL] for i in graph_node[METHODS_LABEL]] 
    # if len(method_names):
    #     content += f"Methods: {', '.join(method_names)}"
    return content


def get_graph_node_str(graph_node):
    content = f"Class {graph_node[CLASS_NAME_LABEL]}\n"
    if graph_node[DOCSTRING_LABEL]:
        content += f"Docstring: {graph_node[DOCSTRING_LABEL]}\n"
        
    # if ATTRIBUTES_LABEL in graph_node and len(graph_node[ATTRIBUTES_LABEL]):
    #     content += f"Attributes: \n"
    #     for attr in graph_node[ATTRIBUTES_LABEL]:
    #         content += f"{attr[ATTRIBUTE_NAME_LABEL]}: {attr[ATTRIBUTES_TYPE_LABEL]}\n"
    
    # method_names = [i[METHOD_NAME_LABEL] for i in graph_node[METHODS_LABEL]] 
    # if len(method_names):
    #     content += f"Methods: {', '.join(method_names)}"
    return content



class NL2CodeTracer(BaseRetriever):
    def __init__(
        self, 
        dataset_name,
        retrieval_distance: int = 1,
        similarity_threshold: float = 0.6,
        base_dir='data_repos/ftlr/datasets',
        chroma_db_dir='indices',
        solutions_file='solution_links_english.txt',
        call_graph_file='method_callgraph.json',
        all_code_files_path='all_code_filenames.txt',
        all_req_file_names='all_req_filenames.txt',
    ):

        self.sem_evaluator = SemanticSimilarityEvaluator(
            embed_model=Settings.embed_model,
            similarity_threshold=similarity_threshold,
        )
        self.retrieval_distance = retrieval_distance
        os.makedirs('results', exist_ok=True)
        self.dataset_name = dataset_name

        self.all_code_files_path = all_code_files_path
        self.all_req_file_names = all_req_file_names

        self.base_dir = base_dir
        self.dataset_dir = f'{base_dir}/{dataset_name}'
        indices_path = f"{chroma_db_dir}/{dataset_name}"
        os.makedirs(indices_path, exist_ok=True)

        self.solutions_file_path = f'{self.dataset_dir}/{dataset_name.lower()}_{solutions_file}'
        self.call_graph_file = f'{dataset_name.lower()}_{call_graph_file}'
        self.class_names2node_map = None
    

    def set_dataset_data(
            self, 
            use_mc=False, 
            use_summary=False,
            use_similar_q=False
        ):
        self.use_mc = use_mc
        self.use_summary = use_summary
        self.use_similar_q = use_similar_q
        
        indexed_nodes, req_nodes = get_dataset_nodes(
            self.dataset_name, 
            'indexed_nodes',
            use_mc=use_mc,
            use_summary=use_summary
        )

        if self.use_similar_q:
            for i in range(len(req_nodes)):
                similar_qs = '\n'.join(req_nodes[i].metadata['similar_queries'][:2])
                req_nodes[i].text += f"\n\nSet of similar requirements as context: \n{similar_qs}"

        self.indexed_nodes = indexed_nodes
        self.req_nodes = req_nodes
        
        self.set_class_node_maps()
        self.set_docs_nxg_and_graph_nodes()
        self.set_solution_links()
    

    def set_solution_links(self):
        solutions = get_solutions(self.solutions_file_path)
        self.req_file_to_node_map = {
            n.metadata['file_name']: solutions[n.metadata['file_name']]
            for n in self.req_nodes
        }
    

    def set_class_node_maps(self):
        self.node_map = {n.hash: n for n in self.indexed_nodes}
        self.class_names2node_map = defaultdict(list)
        for n in self.indexed_nodes:
            self.class_names2node_map[n.metadata[CLASS_NAME_LABEL]].append(n.hash)


    def set_docs_nxg_and_graph_nodes(self):
        self.docs_nxg, self.graph_class_nodes = get_docs_nxg(
            dataset_dir=self.dataset_dir,
            all_code_files_path=self.all_code_files_path,
            callgraph_file_name=self.call_graph_file
        )
    
    async def get_semantic_score(self, query_str, doc_str):
        # print(self.sem_evaluator._embed_model)
        result = await self.sem_evaluator.aevaluate(response=doc_str, reference=query_str)
        return result


    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        # print("Query: ", query_bundle.query_str)
        retrieved_nodes: List[NodeWithScore] = []
        for retriever in self.retrievers:
            retrieved_nodes.extend(retriever.retrieve(query_bundle.query_str))
        
        retrieved_nodes = list({n.node_id: n for n in retrieved_nodes}.values())
        
        class_names = list(set([n.metadata['Class Name'] for n in retrieved_nodes]))
        assert all(n in self.docs_nxg for n in class_names)



        # print("Class names", class_names)

        reachable_classes = list(
            set(sum([
                get_reachable_nodes(self.docs_nxg, class_name, self.retrieval_distance)\
                for class_name in class_names], []
            ))
        )
        node_with_scores = list()
        for c in reachable_classes:
            if c not in self.graph_class_nodes:
                continue
            class_node_str = get_graph_node_str(self.graph_class_nodes[c])
            doc = self.node_map[self.class_names2node_map[c][0]]
            if 'section_summary' in doc.metadata:
                class_node_str += f"\n\nSummary: {doc.metadata['section_summary']}"

            node = Document(
                text=class_node_str, 
                metadata=doc.metadata,
                excluded_llm_metadata_keys=doc.excluded_llm_metadata_keys,
                excluded_embed_metadata_keys=doc.excluded_embed_metadata_keys
            )
            sim_result = asyncio.run(self.get_semantic_score(query_bundle.query_str, class_node_str))
            
            if sim_result.passing:
                node_with_score = NodeWithScore(node=node, score=sim_result.score)
                node_with_scores.append(node_with_score)
        
        # print("Retrieved nodes: ", len(node_with_scores))
        return node_with_scores


    def set_retrivers(
        self,
        types: List[str],
        similarity_top_k: int = 2,
    ):
        self.retrievers: Union[BaseRetriever, List[BaseRetriever]] = []
        if VECTOR_INDEX_RETREIVER in types:
            vector_index = VectorStoreIndex(nodes=self.indexed_nodes, show_progress=True)
            vector_retriever = vector_index.as_retriever(similarity_top_k=similarity_top_k)
            self.retrievers.append(vector_retriever)
        
        if BM25_INDEX_RETREIVER in types:
            bm25_retriever = BM25Retriever.from_defaults(
            nodes=self.indexed_nodes, 
            similarity_top_k=similarity_top_k,
        )
            self.retrievers.append(bm25_retriever)
    

    def evaluate_retrievers(self, retrievers: List[str], both=False):
        self.set_retrivers(retrievers)
        retrievers_str = '_'.join(retrievers)
        
        qes = {f'{retrievers_str}_nl': RetrieverQueryEngine(self)}
        
        if both:
            qes[retrievers_str] = RetrieverQueryEngine(self.retrievers[0])
        
        

        correctness_results = evaluate_response(
            req_nodes=self.req_nodes,
            query_engines=qes,
            solutions_file=self.solutions_file_path,
            dataset_name=self.dataset_name
        )
        mc = '_mc' if self.use_mc else ''
        sm = '_sm' if self.use_summary else ''
        eq = '_eq' if self.use_similar_q else ''
        with open(f'results/{self.dataset_name}_{retrievers_str}{mc}{sm}{eq}_results.json', 'w') as f:
            json.dump(correctness_results, f, indent=4)


    def trace(self):
        self.evaluate_retrievers([VECTOR_INDEX_RETREIVER], both=True)
        self.evaluate_retrievers([BM25_INDEX_RETREIVER], both=True)
        self.evaluate_retrievers([VECTOR_INDEX_RETREIVER, BM25_INDEX_RETREIVER])


In [8]:
datase_name = 'smos'
use_mc = False
use_summary = False
use_similar_q = True

In [9]:
nl2code_tracer = NL2CodeTracer(datase_name)
nl2code_tracer.set_dataset_data(use_mc=use_mc, use_summary=use_summary, use_similar_q=use_similar_q)
# nl2code_tracer.trace()

Number of indexed nodes: 395
Number of requirements nodes: 67
Extracting class info objects


  0%|          | 0/100 [00:00<?, ?it/s]

Number of nodes in the graph: 543
Extracting call graph links
Total: 2354
Present: 2354, Absent: 0
Number of nodes in the graph: 599
Adding method calls
Done!


Creating Documents:   0%|          | 0/599 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Creating Networkx Graph:   0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
from prompts.templates import CLASS_TRACE_TEMPLATE


q = CLASS_TRACE_TEMPLATE.format(requirement=nl2code_tracer.req_nodes[0].text)
print(q)


What are the names of the classes that are related to the following use case requirement?

Nome: Login
Attori: Utente registrato
Descrizione: Permette ad un utente di loggarsi al sistema
Precondizioni:
L’utente non è loggato al sistema L’utente possiede username e password Compila e sottomette il form per il login
Sequenza degli eventi
Utente
Sistema
1.	Verifica che username e password abbiano lunghezza >=5. Se la condizione è rispettata passa al punto due, altrimenti notifica l'errore all'utente.
2.	Cerca nell’archivio se username e password inseriti dall’utente sono presenti tra gli utenti loggabili
3.	Se la ricerca ha avuto successo l’utente viene loggato al sistema
Postcondizioni:
•	Il sistema visualizza l’area di lavoro dell’Utente Registrato •	Interruzione della connessione al server SMOS

Set of similar requirements as context: 
**Requirement 1:** The system shall validate the username and password length to ensure they meet the minimum length requirement of 5 characters before

In [11]:
nl2code_tracer.set_retrivers([VECTOR_INDEX_RETREIVER])
retrieved_nodes: List[NodeWithScore] = []
for retriever in nl2code_tracer.retrievers:
    retrieved_nodes.extend(retriever.retrieve(q))

Generating embeddings: 0it [00:00, ?it/s]

In [21]:
nl2code_tracer.set_retrivers([BM25_INDEX_RETREIVER])
retrieved_nodes: List[NodeWithScore] = []
for retriever in nl2code_tracer.retrievers:
    retrieved_nodes.extend(retriever.retrieve(q))

In [22]:
list(set([n.metadata['Class Name'] for n in retrieved_nodes]))

['ConnessioneWrapper', 'Utility']

In [20]:
nl2code_tracer.req_file_to_node_map[nl2code_tracer.req_nodes[0].metadata['file_name']]

['User',
 'Role',
 'UserListItem',
 'managerUser',
 'ServletLogin',
 'ServletLogout',
 'ServletAlterPersonalDate']

In [23]:
sim_result = await nl2code_tracer.get_semantic_score(q, nl2code_tracer.req_nodes[0].text)
sim_result.score

0.8459037741813271

In [ ]:
# nl2code_tracer.sem_evaluator._embed_model.get_text_embedding(CLASS_TRACE_TEMPLATE.format(requirement=nl2code_tracer.req_nodes[1].text))

In [25]:
datasets = ['smos', 'eTour', 'iTrust', 'eANCI']
for dataset in datasets:
    with open(f'similar_requirements/{dataset}.pkl', 'rb') as f:
        reqs = pickle.load(f)

    print("Total: ", len(reqs))
    print("Before", len([1 for req in reqs if ", ".join(req.metadata['similar_queries']).startswith('Here are')]))
    for req in reqs:
        if ", ".join(req.metadata['similar_queries']).startswith('Here are'):
            req.metadata['similar_queries'] = req.metadata['similar_queries'][1:]

    with open(f'similar_requirements/{dataset}.pkl', 'wb') as f:
        pickle.dump(reqs, f)

Total:  67
Before 67
After 0
Total:  58
Before 58
After 0
Total:  131
Before 131
After 0
Total:  139
Before 139
After 0


In [44]:
from parameters import parse_args
import os
from constants import (
    LLMsMap,
    EmbeddingModelsMap,
)

from api_models import set_llm_and_embed
args = parse_args()

datasets = [
    ('smos', 'bge_m3'), 
    ('eTour', 'bge_large'), 
    ('eANCI' 'bge_m3'), 
    ('iTrust', 'bge_large')
]
use_mcs = [True, False]
use_summaries = [True, False]
use_similar_qs = [True, False]

for dataset, embed_model in datasets:
    llm_name = LLMsMap[args.llm]
    embed_model_name = EmbeddingModelsMap[embed_model]
    print(f"Using LLM: {llm_name}")
    print(f"Using Embedding Model: {embed_model_name}")

    set_llm_and_embed(
        llm_type=args.llm_type,
        llm_name=llm_name,
        embed_model_name=embed_model_name,
    )
    configs = [(mc, sm, eq) for mc in use_mcs for sm in use_summaries for eq in use_similar_qs]
    for config in configs:
        use_mc, use_summary, use_similar_q = config
        nl2code_tracer = NL2CodeTracer(dataset)
        nl2code_tracer.set_dataset_data(
            use_mc=use_mc, 
            use_summary=use_summary, 
            use_similar_q=use_similar_q
        )
        nl2code_tracer.trace()

Using LLM: meta-llama/Meta-Llama-3-70B-Instruct
Using Embedding Model: BAAI/bge-m3


/Users/junaid/Anaconda/anaconda3/envs/ML/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/junaid/Anaconda/anaconda3/envs/ML/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Number of indexed nodes: 897
Number of requirements nodes: 67
Extracting class info objects


  0%|          | 0/100 [00:00<?, ?it/s]

Number of nodes in the graph: 543
Extracting call graph links
Total: 2354
Present: 2354, Absent: 0
Number of nodes in the graph: 599
Adding method calls
Done!


...

  0%|          | 0/100 [00:00<?, ?it/s]

Creating Networkx Graph:   0%|          | 0/100 [00:00<?, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Evaluating for vector_index_retriever_nl


Processing:   0%|          | 0/67 [00:00<?, ?Requirement/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
